In [ ]:
import pandas as pd
import numpy as np
import os
os.chdir(r'D:\workspace\ai_23_work_bj\pandasProject')   # 修改相对路径的位置.
# os.getcwd()

# 解决中文显示问题，下面的代码只需运行一次即可
import matplotlib as plt
plt.rcParams['font.sans-serif'] = ['SimHei']    # 如果是Mac本, 不支持SimHei的时候, 可以修改为 'Microsoft YaHei' 或者 'Arial Unicode MS'
plt.rcParams['axes.unicode_minus'] = False

# 1. Pandas进阶语法_缺失值处理

## 1.1 思路1: 删除缺失值

In [ ]:
# 1. 读取数据.
movie_df = pd.read_csv('./data/movie.csv')
movie_df

In [ ]:
# 2. 查看下数据的介绍.
movie_df.columns        # 所有的列名
movie_df.info()         # 查看数据的 基本信息(列名, 数据类型, 非缺失值数量等)
movie_df.describe()     # 查看数据的 描述性统计信息(均值, 中位数, 标准差等)

In [ ]:
# 3. 删除缺失值
# movie_df.dropna()   # 不会修改原数据, 加入 inplace=True 即可, 默认删: axis=0, 删行, axis=1, 删列.

# movie_df.dropna(axis=0)     # 删行
movie_df.dropna(axis=1)      # 删列

## 1.2 思路2: 填充缺失值

In [ ]:
# 1. 查看源数据
movie_df

In [ ]:
# 2. 判断某列(的某个值)是否有缺失值. 
pd.isnull(movie_df)     # 判断df对象的 每列的 每个值 是否为空(缺失值)
pd.notnull(movie_df)    # 判断df对象的 每列的 每个值 是否不为空(非缺失值)

# 3.判断某列是否是 包含缺失值的列. 
np.all(pd.notnull(movie_df))    # 整列都是True -> 结果是True, 但凡有False -> 结果是False, 说明该列有缺失.

In [ ]:
# 4. 填充缺失值.
# 写法1: 填充固定值.
movie_df.fillna(23).info()

In [ ]:
# 写法2: 填充 每列的平均值.
movie_df['Revenue (Millions)'].fillna(movie_df['Revenue (Millions)'].mean(), inplace=True)
movie_df['Metascore'].fillna(movie_df['Metascore'].mean(), inplace=True)
movie_df.info()

In [ ]:
# 5. for循环的方式, 使用 每列的平均值 来填充各列的缺失值. 
# 5.1 获取每个列名
for col_name in movie_df.columns:
    # 5.2 判断某列是否有缺失值.
    # movie_df[col_name]:  根据列名, 找到 df中的某个列 -> Series对象.
    # pd.notnull(某列数据): 判断该列的每个值是否为非缺失值, True -> 不为空, False -> 为空(缺失值)
    # np.all([True, False...]): 里边的值全部为True -> 结果为True, 只要有一个为False -> 结果为False.
    if np.all(pd.notnull(movie_df[col_name])) == False:
        # 5.3 走到这里, 说明该列有缺失值.
        print(col_name)     # 打印列名
        # 5.4 打印这两列的平均值. 
        print(movie_df[col_name].mean())
        # 5.5 用该列的平均值来填充该列的缺失值.
        movie_df[col_name].fillna(movie_df[col_name].mean(), inplace=True)

In [ ]:
# 6. 查看处理后的结果.
movie_df.info()

## 1.3 思路3: 转换, 然后填充或者删除缺失值

In [ ]:
# 背景: 实际开发中, 不是所有的缺失值都会用NaN来表示, 例如: 可能用 ? 表示, 如何删除这些缺失值呢? 
# 思路: 先转换, 后删除.  即:  ? -> NaN -> 删除.
# 1. 加载数据.
wis = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data")
wis

In [ ]:
# 2. 尝试直接删除缺失值.
wis.dropna()        # ? 不是缺失值, 所以直接删, 删不掉. 

In [ ]:
# 3. 解决上述的问题, ? -> NaN -> 删.            空的三种写法: np.nan, np.NAN, np.NaN
wis.replace('?', np.nan).dropna()

# 2. 数据合并

## 2.1 思路1: concat(), 能行合并, 还是列合并

In [ ]:
# 1. 准备数据. 
df = pd.read_csv('./data/1960-2019全球GDP数据.csv', encoding='gbk')
df

df1 = df[:10]
df1

df2 = df[10:20]
df2

In [ ]:
# 2. 通过 concat() 合并数据.
# 细节: 列合并默认参考 列名.  
new_df = pd.concat([df1, df2], axis=0)  # axis=0, 列合并(垂直合并), axis=1, 行合并(水平合并)
new_df = pd.concat([df1, df2])          # 效果同上, 默认是 列合并

# 细节: 行合并, 默认参考 行索引(值)
new_df = pd.concat([df1, df2], axis=1)  # 行合并
new_df

In [ ]:
# 3. 修改df2的列索引.
df2.index = [0, 1, 2, 3, 4, 11, 12, 13, 14, 15]
df2

In [ ]:
# 4. 再次进行 行合并(参考: 行索引值)
pd.concat([df1, df2], axis=1)

In [ ]:
# 5. 再次进行行合并. 
# 默认是: 满外连接, 即: 左表全集 + 右表全集 + 交集
pd.concat([df1, df2], axis=1, join='outer')    

# 指定为: 内连接, 即: 只要交集
pd.concat([df1, df2], axis=1, join='inner') 


## 2.2 思路2: merge() -> 只能进行 行合并(水平合并)

In [ ]:
# 1. 准备数据集.
df1 = pd.DataFrame({'key1': ['K0', 'K0', 'K1', 'K2'],
                        'key2': ['K0', 'K1', 'K0', 'K1'],
                        'A': ['A0', 'A1', 'A2', 'A3'],
                        'B': ['B0', 'B1', 'B2', 'B3']})

df2 = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                        'key2': ['K0', 'K0', 'K0', 'K0'],
                        'C': ['C0', 'C1', 'C2', 'C3'],
                        'D': ['D0', 'D1', 'D2', 'D3']})

# 2. 查看数据
df1
df2

In [ ]:
# 3. 演示 merge()函数的 默认合并方式.
pd.merge(df1, df2, how='inner', on=['key1', 'key2'])
pd.merge(df1, df2)      # 效果同上, 默认是 inner join, 且参考 同名列进行合并.

In [ ]:
# 4. 内连接, 指定合并字段
pd.merge(df1, df2, how='inner', on='key2')  # 内连接, 关联字段为: key2

In [ ]:
# 5. 外连接, 指定合并字段.
pd.merge(df1, df2, how='outer', on=['key1', 'key2'])    # 满外连接, 关联字段为: key1, key2

# 左外连接 = 左表全集 + 交集
# pd.merge(df1, df2, how='left', on=['key1', 'key2'])

In [ ]:
# 6. merge()函数的其它写法.
df1.merge(df2, how='inner', on='key1')      # 效果同上, 即: df1.merge(df2)

# 回顾: concat写法(),   pd.concat([df1, df2...])        可以同时拼接多个, 既能行合并, 也能列合并, 不能指定合并字段. 默认是: 外连接合并方式.
# df1.concat(df2)     # 错误写法.
pd.concat([df1, df2, df1])

# 3. groupby() 分组聚合 以及 分组过滤

In [ ]:
# 1. 读取数据
df = pd.read_csv('./data/uniqlo.csv')
df

## 3.1 分组聚合

In [ ]:
# agg -> Aggregate(聚合的意思)
# 格式: df.groupby(['分组字段1', '分组字段2'...]).agg({'列名1':'聚合函数名', '列名2':'聚合函数名'...})

In [ ]:
# 1. 场景1: 按照单列分组
df.groupby(['city'])        # DataFrameGroupBy  -> DataFrame分组对象
df.groupby('city')        # 细节: 如果分组列只有1个, 中括号可以省略不写.

In [ ]:
# 2. 场景2: 按照多列分组
df.groupby(['city', 'channel'])             # DataFrameGroupBy  -> DataFrame分组对象
df.groupby(['city', 'channel']).revenue     # SeriesGroupBy  -> Series分组对象

In [ ]:
# 3. 场景3: 如何获取某个分组的数据. 
df.groupby(['city', 'channel']).get_group(('北京', '线下'))     # 1个分组的信息
df.groupby(['city', 'channel']).get_group(('上海', '线上'))     # 另1个分组的信息

In [ ]:
# 4. 场景4: 分组 + 聚合(聚合字段只有1个)
# 需求: 根据 城市 和 销售渠道分组, 计算: 销售金额.
# 写法1: 通用版(掌握)
df.groupby(['city', 'channel']).agg({'revenue':'sum'})      # 返回DataFrame对象

# 写法2: 变形版(了解)
df.groupby(['city', 'channel']).revenue.sum()               # 返回Series对象
df.groupby(['city', 'channel'])['revenue'].sum()            # 返回Series对象
df.groupby(['city', 'channel'])[['revenue']].sum()          # 返回DataFrame对象

In [ ]:
# 5. 场景5: 分组 + 聚合(聚合字段有2个, 聚合操作相同)
# 需求: 根据 城市 和 销售渠道分组, 计算: 销售金额, 订单数量 总和.
# 写法1: 通用版(掌握)
df.groupby(['city', 'channel']).agg({'revenue':'sum', 'order':'sum'})

# 写法2: 变形版(了解)
df.groupby(['city', 'channel'])[['revenue', 'order']].sum()

In [ ]:
# 6. 场景6: 分组 + 聚合(聚合字段有2个, 聚合操作不同)
# 需求: 根据 城市 和 销售渠道分组, 分别计算: 销售金额的平均值, 成本的总和.
df.groupby(['city', 'channel']).agg({
    'revenue': 'mean',
    'unit_cost': 'sum'
})

## 3.2 分组过滤

In [ ]:
# 需求: 按照城市分组, 查询每组销售金额平均值 大于200的全部数据.    即: 算出每组的销售金额, 找到金额大于200的组, 然后显示这些组所有的信息.

# 1.根据城市分组, 计算每个城市的 销售金额的 平均值
df.groupby('city').revenue.mean()       

# 2. 根据城市分组, 查看上海分组的数据.
df.groupby('city').get_group('上海')

# 3. 完成需求.
df.groupby('city').filter(lambda x: x.revenue.mean() > 200)
df.groupby('city').revenue.filter(lambda x: x.mean() > 200)

# 4. 上边的代码等价于 找到  city值为 北京, 南京的数据. 
df.query('city in ["北京", "南京"]')

# 4. 交叉表(了解) 和 透视表(掌握)

In [ ]:
# 1. 交叉表演示: 
# 创建一个示例数据集
data = {
    '性别': ['男', '女', '男', '女', '男', '女', '女', '男'],
    '购买': ['是', '否', '是', '是', '否', '否', '是', '否']
}

df = pd.DataFrame(data)

# 创建交叉表
crosstab = pd.crosstab(df['性别'], df['购买'])
print(crosstab)

In [ ]:
# 2. 用透视表完成上述的需求.
data = {
    '性别': ['男', '女', '男', '女', '男', '女'],
    '购买': ['是', '否', '是', '是', '否', '否'],
    '金额': [100, 150, 200, 130, 160, 120]
}
df = pd.DataFrame(data)
df.pivot_table(index='性别', columns='购买', values='金额' , aggfunc='mean')

In [181]:
# 3. 透视表案例: 优衣库数据集.
# 3.1 读取数据
df = pd.read_csv('./data/uniqlo.csv')
df

# 3.2 需求: 根据城市, 销售渠道分组, 计算 销售金额 总和.
# 写法1: groupby() 分组 聚合 
df.groupby(['city', 'channel']).agg({'revenue': 'sum'})
# df.groupby(['city', 'channel'], as_index=False).agg({'revenue': 'sum'})     # as_index=False 不把分组字段当做行索引, 而是当做列.

# 写法2: pivot_table() 透视表, 作用: 统计分组数据, 简化 分组聚合写法, 指定某一列对另一列的关系
# 参1: 指定分组字段, 参2: 指定分组字段, 参3: 指定聚合字段, 参4: 指定聚合函数
df.pivot_table(index='city', columns='channel', values='revenue', aggfunc='sum')

channel,线上,线下
city,,
上海,114438.09,275383.64
北京,NaN,130458.62
南京,NaN,123150.93
广州,200893.30,117231.19
成都,NaN,208189.86
杭州,NaN,589518.49
武汉,281420.73,308357.05
深圳,NaN,733123.68
西安,30088.01,180686.61
